In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local") \
    .appName("pantheon") \
    .getOrCreate()

In [41]:
stazioneDf = spark.read.format('com.databricks.spark.csv')\
.options(delimiter=';',header='true', inferschema='true').load('datasets/pantheon20190612-stazione.csv')
stazioneDf.take(1)

[Row(id_dato=1811253, data_ora=datetime.datetime(2018, 10, 12, 14, 25, 15), temp1_media=25.99, temp1_min=25.93, temp1_max=26.08, ur1_media=56.69, ur1_min=52.56, ur1_max=58.97, temp1_ur1_n_letture=5, pioggia_mm=0.0, rad W/mq=75.0, rad_n_letture=5, wind_dir=6, wind_dir_n_letture=5, wind_speed_media=0.3, wind_speed_max=15.62, wind_speed_n_letture=25, pressione_mbar=1085.94, pressione_standard_mbar=1118.68, pressione_n_letture=5, rad W/mq array='{}')]

In [40]:
stazioneDf.printSchema()

root
 |-- id_dato: integer (nullable = true)
 |-- data_ora: timestamp (nullable = true)
 |-- temp1_media: double (nullable = true)
 |-- temp1_min: double (nullable = true)
 |-- temp1_max: double (nullable = true)
 |-- ur1_media: double (nullable = true)
 |-- ur1_min: double (nullable = true)
 |-- ur1_max: double (nullable = true)
 |-- temp1_ur1_n_letture: integer (nullable = true)
 |-- pioggia_mm: double (nullable = true)
 |-- rad W/mq: double (nullable = true)
 |-- rad_n_letture: integer (nullable = true)
 |-- wind_dir: integer (nullable = true)
 |-- wind_dir_n_letture: integer (nullable = true)
 |-- wind_speed_media: double (nullable = true)
 |-- wind_speed_max: double (nullable = true)
 |-- wind_speed_n_letture: integer (nullable = true)
 |-- pressione_mbar: double (nullable = true)
 |-- pressione_standard_mbar: double (nullable = true)
 |-- pressione_n_letture: integer (nullable = true)
 |-- rad W/mq array: string (nullable = true)



In [28]:
# con questa opzione della Spark Session impostata a True, non c'è più bisogno di usare show per vedere i dataframe;
# inoltre li posso vedere in modo non sballato
spark.conf.set("spark.sql.repl.eagerEval.enabled",True)

In [29]:
stazioneDf

id_dato,data_ora,temp1_media,temp1_min,temp1_max,ur1_media,ur1_min,ur1_max,temp1_ur1_n_letture,pioggia_mm,rad W/mq,rad_n_letture,wind_dir,wind_dir_n_letture,wind_speed_media,wind_speed_max,wind_speed_n_letture,pressione_mbar,pressione_standard_mbar,pressione_n_letture,rad W/mq array
1811253,2018-10-12 14:25:15,25.99,25.93,26.08,56.69,52.56,58.97,5,0.0,75.0,5,6,5,0.3,15.62,25,1085.94,1118.68,5,{}
1811269,2018-10-12 14:30:12,25.81,25.35,26.03,57.25,49.95,61.32,5,0.0,104.1,5,6,5,0.91,5.41,75,1087.05,1119.79,5,{}
1811283,2018-10-12 14:35:11,24.34,23.88,24.98,52.91,51.54,54.04,5,0.0,234.4,5,52,5,3.89,9.43,322,1085.7,1118.44,5,{}
1811284,2018-10-12 14:40:13,23.82,23.76,23.89,54.17,53.44,54.91,5,0.0,247.2,5,44,5,2.37,8.19,196,1088.63,1121.37,5,{}
1811304,2018-10-12 14:45:13,23.74,23.66,23.86,55.22,53.6,56.69,5,0.0,206.6,5,72,5,1.93,5.7,160,1086.11,1118.85,5,{}
1811314,2018-10-12 14:50:10,23.84,23.76,23.92,57.18,55.32,58.4,5,0.0,250.3,5,68,5,1.3,3.41,108,1086.41,1119.15,5,{}
1811315,2018-10-12 14:55:13,23.92,23.84,23.97,56.15,55.42,56.99,5,0.0,307.5,5,42,5,2.88,8.1,239,1086.35,1119.09,5,{}
1811334,2018-10-12 15:00:15,24.45,24.1,24.71,54.74,52.34,57.76,5,0.0,360.5,5,72,5,2.63,6.32,218,1086.35,1119.09,5,{}
1811345,2018-10-12 15:05:11,24.37,24.3,24.43,54.01,53.14,54.93,5,0.0,307.5,5,53,5,2.91,7.23,241,1086.29,1119.03,5,{}
1811346,2018-10-12 15:10:13,24.71,24.58,24.86,52.08,48.88,54.2,5,0.0,388.4,5,69,5,3.13,8.12,259,1086.29,1119.03,5,{}


## Calcolo somma di pioggia ogni mese

In [31]:
stazioneDf.select('data_ora', 'pioggia_mm')

data_ora,pioggia_mm
2018-10-12 14:25:15,0.0
2018-10-12 14:30:12,0.0
2018-10-12 14:35:11,0.0
2018-10-12 14:40:13,0.0
2018-10-12 14:45:13,0.0
2018-10-12 14:50:10,0.0
2018-10-12 14:55:13,0.0
2018-10-12 15:00:15,0.0
2018-10-12 15:05:11,0.0
2018-10-12 15:10:13,0.0


In [32]:
from pyspark.sql.functions import month, year
from pyspark.sql.functions import sum, avg
# fare lo show() non crea effettivamente l'oggetto ma lo mostra solamente, quindi se voglio creare un oggetto ed
# untilizzarlo, non devo usare lo show in un punto intermedio
df1 = stazioneDf.select(year('data_ora').alias("year"), month('data_ora').alias("month"), "pioggia_mm")\
.groupBy("year", "month").agg(sum("pioggia_mm").alias("total_rain")).orderBy("year", "month")

#df2 = stazioneDf.select(year('data_ora').alias("year"), month('data_ora').alias("month"))

#df_rain = df2.join(df1, 'month').show()

#.groupBy("month").agg(avg("year")).show()
#print(df1)

In [33]:
df1

year,month,total_rain
2018,10,13.15999999999998
2018,11,40.67999999999985
2018,12,9.079999999999988
2019,1,23.599999999999905
2019,2,16.839999999999968
2019,3,1.4000000000000004
2019,4,13.159999999999961
2019,5,37.5599999999997
2019,6,0.0


## Rilevamento di anomalie

In [34]:
from pyspark.sql import SparkSession
df1.createOrReplaceTempView("table1")
# spark.sql("SELECT year, month, sum(pioggia_mm) AS total_rain FROM table1 GROUP BY year, month")

In [35]:
# job che rileva se per un certo periodo di tempo ci sono valori nulli su una o più colonne, manda un alert
# o comunque da' TRUE. Questo può essere utile per capire se i sensori si sono guastati

In [36]:
# questa cella serve ad individuare i blocchi del dataframe accomunati da una caratteristica comune, ad esempio in questo caso
# vogliamo raggruppare tutti i blocchi con radiazione nulla e tutti quelli con radiazione non nulla
# la funzione lag serve , data una certa riga , a selezionare la riga che si trova ad un certo offset da questa, nel nostro caso 1
# usiamo lag per selezionare la riga subito sotto alla riga di inizio di un blocco 
# coalesce serve per selezionare, data una lista di colonne, la prima colonna non nulla
# count considera nel conteggio solo le celle non vuote, così può essere utile per individuare una sottocolonna con tutti valori non nulli
import pyspark.sql.functions as F
from pyspark.sql.window import Window
nullRadSlot = stazioneDf
nullRadSlot = nullRadSlot.withColumn("isnull", F.when(nullRadSlot['rad W/mq'].isNull(), True).otherwise(False))
nullRadSlot = nullRadSlot.withColumn("lag_isnull", F.lag(nullRadSlot["isnull"],1).over(Window.orderBy(nullRadSlot["data_ora"])))
nullRadSlot = nullRadSlot.withColumn("change", F.coalesce(nullRadSlot["isnull"]!=nullRadSlot["lag_isnull"],F.lit(False)))
nullRadSlot = nullRadSlot.withColumn("block", F.sum(nullRadSlot["change"].cast("int")).over(Window.orderBy(nullRadSlot["data_ora"])))\
  .groupBy("block")\
  .agg(F.min(nullRadSlot["data_ora"]).alias('mindata'),
    F.max(nullRadSlot["data_ora"]).alias('maxdata'),
    (F.count(nullRadSlot['rad W/mq'])==0).alias('blocco_isnull'))

In [37]:
nullRadSlot

block,mindata,maxdata,blocco_isnull
0,2018-10-12 14:25:15,2019-05-06 08:55:28,false
1,2019-05-06 13:40:10,2019-06-12 09:25:15,true


In [38]:
timeFmt = "yyyy-MM-dd'T'HH:mm:ss.SSS"
timeDiff = (F.unix_timestamp('maxdata', format=timeFmt)
            - F.unix_timestamp('mindata', format=timeFmt))
nullRadSlot = nullRadSlot.withColumn("Duration", timeDiff)

In [39]:
nullRadSlot

block,mindata,maxdata,blocco_isnull,Duration
0,2018-10-12 14:25:15,2019-05-06 08:55:28,false,17778613
1,2019-05-06 13:40:10,2019-06-12 09:25:15,true,3181505


## Calcolo temperatura media ed umidità settimanale

In [44]:
from pyspark.sql.functions import weekofyear
stazioneDf.select(weekofyear('data_ora'), 'temp1_media', 'ur1_media')

weekofyear(data_ora),temp1_media,ur1_media
41,25.99,56.69
41,25.81,57.25
41,24.34,52.91
41,23.82,54.17
41,23.74,55.22
41,23.84,57.18
41,23.92,56.15
41,24.45,54.74
41,24.37,54.01
41,24.71,52.08


In [53]:
df2 = stazioneDf.select(where(stazioneDf.year('data_ora') == 2, stazioneDf.year('data_ora') + 1).alias("year"), \
                        weekofyear('data_ora').alias("week"), \
                        "temp1_media", \
                        "ur1_media")\
.groupBy("week").agg(avg("temp1_media").alias("mean_temp1"),\
                              avg("ur1_media").alias("mean_ur1")).orderBy("week")

In [54]:
df2

week,mean_temp1,mean_ur1
1,4.075252976190477,61.58029761904763
2,3.711937406855441,72.22089418777941
3,5.784401197604781,79.48673652694613
4,3.8717168674698805,80.87670180722884
5,5.550687593423019,90.52663677130032
6,7.322846497764531,74.13402384500745
7,8.061223880597026,53.742253731343304
8,6.788776119402983,67.70213432835823
9,9.962324888226531,59.35160953800301
10,10.296855439642323,82.48035767511178
